# SDK v1 code

In [ ]:
import pandas as pd
import numpy as np
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig
from azureml.core.compute import AmlCompute,ComputeTarget, ComputeInstance
from azureml.exceptions import ComputeTargetException
from azureml.core.datastore import Datastore
from azureml.widgets import RunDetails
from azureml.core.environment import CondaDependencies

In [ ]:
ws = Workspace.from_config(path='../../config/config.json')

## Environment

In [ ]:
env=Environment.from_pip_requirements("conv_sum",  '../config/requirements.txt')
env.register(ws)

## Compute

In [ ]:
from ComputeManagement import create_cluster, create_instance, delete_compute

In [ ]:
cc=create_cluster(
    workspaceRef=ws,
    name="q34",
    vmSize="Standard_DS3_v2",
    minNodes=0,
    maxNodes=4,
    idleTime=180
)

## Training

In [ ]:
# training_script_config = ScriptRunConfig(
#     source_directory = 'src',
#     script = 'training_script.py',
#     arguments=['--data',___],
#     environment = env,
#     compute_target = cc
# )
# experiment = Experiment(
#     workspace = ws,
#     name="maiden_experiment"
# )
# run = experiment.submit(config=training_script_config, tags=[])

# RunDetails(run).show()
# run.wait_for_completion(show_output=True)

In [ ]:
import yfinance as yf

In [ ]:
sensexTickerYFinance = ['HDFCLIFE.NS, NESTLEIND.NS, KOTAKBANK.NS, INDUSINDBK.NS, TATASTEEL.NS, ITC.NS, ONGC.NS, TITAN.NS, ULTRACEMCO.NS, BAJAJFINSV.NS, BAJFINANCE.NS, BRITANNIA.NS, BAJAJ-AUTO.NS, COALINDIA.NS, BHARTIARTL.NS, TATACONSUM.NS, LTI.NS, CIPLA.NS, MARUTI.NS, ICICIBANK.NS, APOLLOHOSP.NS, NTPC.NS, HEROMOTOCO.NS, HINDALCO.NS, WIPRO.NS, TCS.NS, ADANIENT.NS, MM.NS, TECHM.NS, RELIANCE.NS']
stock_data = yf.download(tickers=sensexTickerYFinance, start='2000-01-01', end='2022-12-31', interval='1mo')

In [ ]:
stock_data.loc[:,'Adj Close']

 TODO
1. Read about SOTA for stock price prediction and what determines success of model trying to predict price
2. Choose stocks to monitor - Nifty 50
3. Build as below

Think of a common use-case where data would update regularly and model would shift
1. Stock price prediction
2. Automatic data retrieval using API to store into Azure storage
3. Automatic model training at intervals depending on error rate

Tie everything up in a RL portfolio optimization application

In [ ]:
import yfinance as yf

In [ ]:
tickerData= yf.download(tickers="RELIANCE.NS", start="2022-01-01", end="2023-01-10", period="1d")
tickerData['Date'] = [str(x)[:10] for x in tickerData.index]
tickerData['Ticker'] = "RELIANCE.NS"
tickerDataToPersist = list(tickerData.transpose().to_dict().values())

In [ ]:
from src.TickerData import query, download

In [ ]:
download(ticker="RELIANCE.NS", start="2022-12-01",end="2023-01-10", period="1d")

In [ ]:
train_data.to_csv()

In [ ]:
train_data = query(ticker="RELIANCE.NS", start="2022-12-01",end="2023-01-10")

In [ ]:
train_data

In [ ]:
import pandas as pd

In [ ]:
import torch

In [ ]:
ticker="RELIANCE.NS"

In [ ]:
train_data = train_data[f"{ticker}_Close"]

In [ ]:
import pandas as pd

In [ ]:
pd.Series(train_data).to_csv('./data/ril.csv')

In [ ]:
import numpy as np

In [ ]:
def training_data(series, loookaheadSize=5):
    X,y = [],[]
    for i in np.arange(5,len(series)-1):
        X.append(series[i-loookaheadSize:i])
        y.append(series[i+1])
    X = np.array(X)
    y = np.array(y)
    X = X.reshape(len(series)-loookaheadSize-1,1,5)
    y=y.reshape(-1,1)

    train_dataset = torch.utils.data.TensorDataset(torch.from_numpy(X), torch.from_numpy(y))

    return train_dataset

In [ ]:
tx=training_data(train_data)

In [ ]:
next(iter(tx))

In [ ]:
torch.save(tx,'txx.pt')

In [ ]:
typ = torch.load('txx.pt')

In [ ]:
next(iter(typ))

In [ ]:
next(iter(tx))

In [ ]:
import numpy as np

In [ ]:
np.array([0.0026]).shape

In [ ]:
from datetime import datetime
str(datetime.now().date())

# SDK v2 code

In [118]:
import json
import numpy as np
from azure.ai.ml import MLClient, Input, Output, command
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml.entities import AmlCompute, Environment, Model, Data, CodeConfiguration, ManagedOnlineEndpoint, ManagedOnlineDeployment
from azure.ai.ml.constants import AssetTypes
from datetime import datetime

## Init

In [3]:
with open('../config/config.json','r') as f:
    configs=json.loads(f.read())

subscription_id, resource_group, workspace = configs['subscription_id'], configs['resource_group'], configs['workspace_name']

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

ml_client = MLClient(
    credential, subscription_id, resource_group, workspace
)

## Compute creation

In [246]:
def create_train_cluster(
    TargetName = "cpu-cluster",
    computeSize="STANDARD_DS2_V2",
    minInstances=1,
    maxInstances=2,
    idleTime=180,
    ):

    try:
        compute = ml_client.compute.get(TargetName)
    except Exception:
        print("Creating a new cpu compute target...")
        # Let's create the Azure ML compute object with the intended parameters
        compute = AmlCompute(
            name=TargetName,
            type="amlcompute",
            size=computeSize,
            min_instances=minInstances,
            max_instances=maxInstances,
            idle_time_before_scale_down=idleTime,
        )
        # Now, we pass the object to MLClient's create_or_update method
        ml_client.begin_create_or_update(compute)    
    return compute

In [249]:
compute = create_train_cluster(TargetName="cluster2")

Creating a new cpu compute target...


In [250]:
compute

AmlCompute({'type': 'amlcompute', 'created_on': None, 'provisioning_state': None, 'provisioning_errors': None, 'name': 'cluster2', 'description': None, 'tags': {}, 'properties': {}, 'id': None, 'Resource__source_path': None, 'base_path': '/Users/anupam/Documents/Codebase/MLOps_stock_prediction/notebooks', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x154a83390>, 'resource_id': None, 'location': 'eastus', 'size': 'STANDARD_DS2_V2', 'min_instances': 1, 'max_instances': 2, 'idle_time_before_scale_down': 180, 'identity': None, 'ssh_public_access_enabled': None, 'ssh_settings': None, 'network_settings': None, 'tier': None, 'subnet': None})

## Environment creation

In [15]:
dependencies_dir = "../config"
os.makedirs(dependencies_dir, exist_ok=True)

In [212]:
%%writefile {dependencies_dir}/conda.yml
name: model-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - numpy=1.21.2
  - pip=21.2.4
  - scipy=1.7.1
  - pandas>=1.1,<1.2
  - pip:
    - inference-schema[numpy-support]==1.3.0
    - xlrd==2.0.1
    - mlflow== 1.26.1
    - azureml-mlflow==1.42.0
    - yfinance==0.2.4
    - pymongo==4.3.3
    - torch==1.13.1
    - scikit-learn==1.2.0
    - werkzeug==0.16.1
    - opencensus==0.11.1
    - markupsafe==2.0.1
    - opencensus-ext-azure==1.1.8


Overwriting ../config/conda.yml


In [178]:
def register_env(
    dependencies_dir,
    envName = "stock-pred"
    ):

    env = Environment(
        name=envName,
        description="Custom environment for creating MLOps project for stock prediction",
        conda_file=os.path.join(dependencies_dir, "conda.yml"),
        image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
        version="0.1.6",
    )
    env = ml_client.environments.create_or_update(env)

    print(
        f"Environment with name {env.name} is registered to workspace, the environment version is {env.version}"
    )
    return env

In [179]:
env = register_env(dependencies_dir=dependencies_dir)

..Environment with name stock-pred is registered to workspace, the environment version is 0.1.6


...

## Upload data to Azure

In [43]:
my_path = '../data/ril.csv'

my_data = Data(
    path=my_path,
    type=AssetTypes.URI_FILE,
    description="ril_stock_data",
    name="ril",
    version='1'
)

ml_client.data.create_or_update(my_data)

Data({'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'ril', 'description': 'ril_stock_data', 'tags': {}, 'properties': {}, 'id': '/subscriptions/5d2e45e0-cd7b-4338-b279-455fa4a4c42d/resourceGroups/RG/providers/Microsoft.MachineLearningServices/workspaces/AzureMLWorkspace/data/ril/versions/1', 'Resource__source_path': None, 'base_path': '/Users/anupam/Documents/Codebase/MLOps_stock_prediction/notebooks', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x10ce04b10>, 'serialize': <msrest.serialization.Serializer object at 0x10ce06410>, 'version': '1', 'latest_version': None, 'path': 'azureml://subscriptions/5d2e45e0-cd7b-4338-b279-455fa4a4c42d/resourcegroups/RG/workspaces/AzureMLWorkspace/datastores/workspaceblobstore/paths/LocalUpload/3e91b5da3fd7435efb8a68549a7c027a/ril.csv', 'datastore': None})

In [20]:
ril_data = ml_client.data.get(name='ril', version=1)

In [54]:
ril_data.path

'azureml://subscriptions/5d2e45e0-cd7b-4338-b279-455fa4a4c42d/resourcegroups/RG/workspaces/AzureMLWorkspace/datastores/workspaceblobstore/paths/LocalUpload/3e91b5da3fd7435efb8a68549a7c027a/ril.csv'

## Training

In [26]:
src_dir = "../src/"

In [255]:
%%writefile {src_dir}Train.py
import os
import argparse
import pandas as pd
import torch
from sklearn.preprocessing import  MinMaxScaler
from sklearn.model_selection import train_test_split
import logging
import mlflow
import pickle
import numpy as np

def series_to_tensors(series, lookaheadSize=5):

    X,y = [],[]
    for i in np.arange(5,len(series)-1):
        X.append(series[i-lookaheadSize:i])
        y.append(series[i+1])
    X = np.array(X)
    y = np.array(y)
    X = X.reshape(len(series)-lookaheadSize-1,1,5)
    y=y.reshape(-1,1)

    dataset = torch.utils.data.TensorDataset(torch.from_numpy(X), torch.from_numpy(y))

    return dataset

def dataprep(args):

    stockData = pd.read_csv(args.data, index_col='Unnamed: 0')
    stock_train_df, stock_test_df = train_test_split(stockData, test_size=args.test_train_ratio)

    print(stock_train_df)

    # Instead of this use LayerNorm or BatchNorm in the neural net
    scaler = MinMaxScaler().fit(stock_train_df)
    stock_train_df = scaler.transform(stock_train_df)
    stock_test_df = scaler.transform(stock_test_df)

    train_tensors = series_to_tensors(stock_train_df)
    test_tensors = series_to_tensors(stock_test_df)

    return scaler, train_tensors, test_tensors

class lstm_model(torch.nn.Module):

    def __init__(self):
        super(lstm_model, self).__init__()
        self.lstm1=torch.nn.LSTM(batch_first=True, input_size=5, hidden_size=1)
        self.out=torch.nn.Linear(1,1)

    def forward(self, x, hidden=None):
        x, hidden = self.lstm1(x)
        x = self.out(x)
        return x.flatten()

def train(trainset):

    seq_model = lstm_model()
    optim = torch.optim.Adam(lr = 0.0001, params=seq_model.parameters())

    epochs = 10

    for epoch in np.arange(epochs):

        Loss=0

        for data in trainset:

            feats, target = data
            optim.zero_grad()

            y_p = seq_model(feats.float())
            loss = torch.nn.functional.mse_loss(y_p.float(), target.float())

            loss.backward()
            optim.step()
            Loss += loss.item()

        print(f"Epoch: {epoch}, loss: {Loss}")
    return seq_model

def main():
    """Main function of the script."""

    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--registered_model_name", type=str, help="model name")
    parser.add_argument("--data", type=str, help="Path to input data")
    parser.add_argument("--local_model_name", type=str, required=True, default=0.25)
    parser.add_argument("--test_train_ratio", type=float, required=False, default=0.25)

    args = parser.parse_args()
    
    # Load Scaler object later and send it for scaling data

    scaler, trainset, _ = dataprep(args)

    trainedModel = train(trainset)

    print(os.getcwd())

    pickle.dump(scaler, open('./outputs/scaler.pkl','wb'))
    model_file = f"./outputs/{args.local_model_name}.pth"
    torch.save(trainedModel.state_dict(), model_file)

if __name__ == "__main__":
    main()

Overwriting ../src/Train.py


## Run training job

In [22]:
def version_iter(n=20):
    i=0
    for i in np.arange(21,50):
        yield i
x = iter(version_iter())
next(x)

21

In [256]:
local_model_name = f"modelstock_pred_{str(datetime.now().date())}"
registered_model_name = "stock_pred_v1"

job = command(   
    name=f"stock_pred_job_{next(x)}",
    inputs={
        "data": Input(type=AssetTypes.URI_FILE, mode="ro_mount", path=ril_data.path),
        "test_train_ratio": 0.25,
        "registered_model_name":registered_model_name,
        "local_model_name":local_model_name
        },
    code="../src/",  # location of source code
    command="python Train.py --data ${{inputs.data}} --test_train_ratio ${{inputs.test_train_ratio}} --local_model_name ${{inputs.local_model_name}} --registered_model_name ${{inputs.registered_model_name}}",
    environment=env,
    compute=compute.name,
    experiment_name="train_model_stock_price_prediction",
    display_name="stock_price_prediction",
)

ml_client.create_or_update(job)

Uploading src (0.01 MBs): 100%|██████████| 12353/12353 [00:01<00:00, 9359.10it/s] 




Experiment,Name,Type,Status,Details Page
train_model_stock_price_prediction,stock_pred_job_34,command,Starting,Link to Azure Machine Learning studio


## Register the trained model

In [257]:
local_model_name=f"modelstock_pred_{str(datetime.now().date())}"

In [258]:
local_model_name

'modelstock_pred_2023-01-25'

In [269]:
model = Model(
        path=f"azureml://jobs/{job.name}/outputs/artifacts/paths/outputs/",
        name="model-path",
        description="Model created from run.",
        type="custom_model",
    )

In [270]:
ml_client.models.create_or_update(model)

Model({'job_name': 'stock_pred_job_34', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'model-path', 'description': 'Model created from run.', 'tags': {}, 'properties': {}, 'id': '/subscriptions/5d2e45e0-cd7b-4338-b279-455fa4a4c42d/resourceGroups/RG/providers/Microsoft.MachineLearningServices/workspaces/AzureMLWorkspace/models/model-path/versions/1', 'Resource__source_path': None, 'base_path': '/Users/anupam/Documents/Codebase/MLOps_stock_prediction/notebooks', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x154eb96d0>, 'serialize': <msrest.serialization.Serializer object at 0x154eb92d0>, 'version': '1', 'latest_version': None, 'path': 'azureml://subscriptions/5d2e45e0-cd7b-4338-b279-455fa4a4c42d/resourceGroups/RG/workspaces/AzureMLWorkspace/datastores/workspaceartifactstore/paths/ExperimentRun/dcid.stock_pred_job_34/outputs', 'datastore': None, 'utc_time_created': None, 'flavors': None, 'arm_type': 'model_version', 'type': 'custom_model'}

## Deployment

In [273]:
%%writefile {src_dir}/Deployment.py
import pandas as pd
import numpy as np
import pickle
import torch
import os
import logging
import json

def init():
    
    global model
    global scaler

    class lstm_model(torch.nn.Module):

        def __init__(self):
            super(lstm_model, self).__init__()
            self.lstm1=torch.nn.LSTM(batch_first=True, input_size=5, hidden_size=1)
            self.out=torch.nn.Linear(1,1)

        def forward(self, x, hidden=None):
            x, hidden = self.lstm1(x)
            x = self.out(x)
            return x.flatten()

    scalerpath = os.path.join(
    os.getenv("AZUREML_MODEL_DIR"), "outputs/scaler.pkl")
    print(scalerpath)
    # deserialize the model file back into a sklearn model
    with open(scalerpath, 'rb') as f:
        scaler = pickle.load(f)

    modelpath = os.path.join(
    os.getenv("AZUREML_MODEL_DIR"), "outputs/modelstock_pred_2023-01-25.pth")    
    print(modelpath)
    model = lstm_model()
    model = model.load_state_dict(torch.load(modelpath))

def run(raw_data):

    data = json.loads(raw_data)["data"]
    data = np.array(data)
    scaled_data = scaler.transform(data)

    result = model.predict(scaled_data)

    return result.tolist()

Overwriting ../src//Deployment.py


In [262]:
# Create endpoint
online_endpoint_name = "modeldir-logged-init"

endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is a sample online endpoint"
)

ml_client.begin_create_or_update(endpoint)

In [266]:
ml_client.online_endpoints.get(name=online_endpoint_name).provisioning_state

'Succeeded'

In [271]:
# Get model
modeldir = ml_client.models.get(name="model-path", version="1")

In [132]:
## Create deployment only after endpoint has provisioned

In [274]:
# Deployment script
code_config = CodeConfiguration(
        code=src_dir, scoring_script="Deployment.py"
    )

# Create deployment
yellow_deployment = ManagedOnlineDeployment(
    name="yellow",
    endpoint_name=online_endpoint_name,
    model=modeldir,
    environment=Environment(
            conda_file=f"./{dependencies_dir}/conda.yml",
            image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04"),
    code_configuration=code_config,
    instance_type="Standard_DS2_v2",
    instance_count=1,
)
# create the deployment:
ml_client.begin_create_or_update(yellow_deployment)
# blue deployment takes 100 traffic
# endpoint.traffic = {"yellow": 100}
# ml_client.begin_create_or_update(endpoint)

Check: endpoint modeldir-logged-init exists
Uploading src (0.01 MBs): 100%|██████████| 12645/12645 [00:01<00:00, 9572.42it/s]


data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


.................................................................................

In [277]:
ml_client.online_deployments.get_logs(
    name="yellow", endpoint_name=online_endpoint_name, lines=50
)

'Instance status:\nSystemSetup: Succeeded\nUserContainerImagePull: Succeeded\nModelDownload: Succeeded\nUserContainerStart: Succeeded\n\nContainer events:\nKind: Pod, Name: Pulling, Type: Normal, Time: 2023-01-25T03:40:18.439511Z, Message: Start pulling container image\nKind: Pod, Name: Downloading, Type: Normal, Time: 2023-01-25T03:40:18.63363Z, Message: Start downloading models\nKind: Pod, Name: Pulled, Type: Normal, Time: 2023-01-25T03:40:33.912731Z, Message: Container image is pulled successfully\nKind: Pod, Name: Downloaded, Type: Normal, Time: 2023-01-25T03:40:33.912731Z, Message: Models are downloaded successfully\nKind: Pod, Name: Created, Type: Normal, Time: 2023-01-25T03:40:34.098431Z, Message: Created container inference-server\nKind: Pod, Name: Started, Type: Normal, Time: 2023-01-25T03:40:34.243024Z, Message: Started container inference-server\nKind: Pod, Name: ContainerReady, Type: Normal, Time: 2023-01-25T03:40:53.777998255Z, Message: Container is ready\n\nContainer logs

## Test the endpoint

In [306]:
%%writefile inputs.json
{"data1":"24.53","data2":"26.544"}

Overwriting inputs.json


In [307]:
# test the endpoint (the request will route to blue deployment as set above)
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="yellow",
    request_file="inputs.json",
)

Exception: 'data'

In [299]:
import json
json.loads('{"data":"24"}')

{'data': '24'}